In [104]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.under_sampling import RandomUnderSampler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

In [2]:
#import data in data frame
subbmission = pd.read_csv('./sample_submission_ejm25Dc.csv')
data = pd.read_excel('./Train/train_Data.xlsx')
data_test = pd.read_excel('./test_Data.xlsx')

In [194]:
#merging data into one large file to preprocess.
big_data = data.append(data_test)

In [195]:
#find a optimise way to do this 
big_data['PaymentMode']=big_data['PaymentMode'].str.replace('PDC_E', 'PDC', regex=True)
big_data['PaymentMode']=big_data['PaymentMode'].str.replace('Cheque', 'PDC', regex=True)

In [224]:
#filling zip code with ranking as its categorical data , ranking is decided on basis of total disburst ammount of each zip code.

zip_ranking =big_data.groupby('ZiPCODE')['DisbursalAmount'].sum().reset_index()

zip_ranking['Ranking'] = zip_ranking['DisbursalAmount'].rank(ascending=False)

ranking_for_zipcode = zip_ranking.set_index('ZiPCODE').to_dict()['Ranking']

big_data['ZiPCODE'] = big_data['ZiPCODE'].map(ranking_for_zipcode)

In [225]:
big_data['ZiPCODE'].isnull().sum()

0

In [226]:
big_data['ZiPCODE'].fillna(0,inplace=True)

In [227]:
big_data['ZiPCODE'].isnull().sum()

0

In [2]:
X = big_data[['ID','Frequency', 'InstlmentMode', 'LoanStatus', 'PaymentMode',
       'Tenure', 'AssetCost', 'AmountFinance',
       'DisbursalAmount', 'EMI', 'DisbursalDate', 'MaturityDAte', 'AuthDate',
     'LTV', 'SEX', 'AGE',
       'MonthlyIncome', 'ZiPCODE', 'Top-up Month']]

NameError: name 'big_data' is not defined

In [269]:
X.isnull().sum()

ID                     0
Frequency              0
InstlmentMode          0
LoanStatus             0
PaymentMode            0
Tenure                 0
AssetCost              0
AmountFinance          0
DisbursalAmount        0
EMI                    0
DisbursalDate          0
MaturityDAte           1
AuthDate               0
ManufacturerID         2
SupplierID             0
LTV                    0
SEX                   64
AGE                   66
MonthlyIncome        248
ZiPCODE                0
Top-up Month       14745
dtype: int64

In [270]:
X.groupby(['AGE'])['MonthlyIncome'].transform('mean')

0        47425.876345
1        44413.691817
2        45098.437186
3        45348.080927
4        46096.110977
             ...     
14740             NaN
14741             NaN
14742             NaN
14743             NaN
14744             NaN
Name: MonthlyIncome, Length: 143400, dtype: float64

In [271]:
X['AGE'].mean()

40.62054362537849

In [272]:
X['AGE'].fillna(40.6,inplace=True)

C:\Users\rushi\anaconda3\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [273]:
X['MonthlyIncome'].fillna(X.groupby(['AGE'])['MonthlyIncome'].transform('mean'),inplace=True)

In [274]:
X.SEX.value_counts()

M    136273
F      7063
Name: SEX, dtype: int64

In [275]:
X['SEX'].fillna('M',inplace=True)

In [276]:
X.ManufacturerID.value_counts()

1046.0    28391
1186.0    25525
1062.0    20026
1568.0    17503
1060.0    17335
1019.0    16926
1187.0    12895
1049.0     3524
3473.0      819
2608.0      431
2965.0       20
2721.0        2
2733.0        1
Name: ManufacturerID, dtype: int64

In [277]:
X['ManufacturerID'].fillna(1046.0,inplace=True)

In [278]:
X.MaturityDAte.value_counts()

2018-06-10    2788
2017-06-10    2735
2017-12-10    2691
2019-06-10    2666
2021-06-10    2569
              ... 
2015-02-08       1
2011-03-18       1
2014-01-26       1
2022-03-03       1
2013-12-22       1
Name: MaturityDAte, Length: 982, dtype: int64

In [279]:
X['MaturityDAte'].fillna('2018-06-10',inplace=True)

In [280]:
X['MaturityDAte']=pd.to_datetime(X['MaturityDAte'])


<ipython-input-280-8676ac7c13ba>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['MaturityDAte']=pd.to_datetime(X['MaturityDAte'])


In [281]:
X.dtypes

ID                          int64
Frequency                  object
InstlmentMode              object
LoanStatus                 object
PaymentMode                object
Tenure                      int64
AssetCost                   int64
AmountFinance             float64
DisbursalAmount           float64
EMI                       float64
DisbursalDate      datetime64[ns]
MaturityDAte       datetime64[ns]
AuthDate           datetime64[ns]
ManufacturerID            float64
SupplierID                  int64
LTV                       float64
SEX                        object
AGE                       float64
MonthlyIncome             float64
ZiPCODE                   float64
Top-up Month               object
dtype: object

In [282]:
test

,ID,Top-up Month
0,4,> 48 Months
1,5,36-48 Months
2,6,> 48 Months
3,25,24-30 Months
4,119,> 48 Months
...,...,...
14740,143396,36-48 Months
14741,143397,36-48 Months
14742,143398,36-48 Months
14743,143399,No Top-up Service


In [283]:
X['DisbursalDate_day']=X['DisbursalDate'].dt.day
X['DisbursalDate_month']=X['DisbursalDate'].dt.month
X['DisbursalDate_year']=X['DisbursalDate'].dt.year


X['MaturityDAte_day']=X['MaturityDAte'].dt.day
X['MaturityDAte_month']=X['MaturityDAte'].dt.month
X['MaturityDAte_year']=X['MaturityDAte'].dt.year

X['AuthDate_day']=X['AuthDate'].dt.day
X['AuthDate_month']=X['AuthDate'].dt.month
X['AuthDate_year']=X['AuthDate'].dt.year

<ipython-input-283-92ce15666475>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['DisbursalDate_day']=X['DisbursalDate'].dt.day
<ipython-input-283-92ce15666475>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['DisbursalDate_month']=X['DisbursalDate'].dt.month
<ipython-input-283-92ce15666475>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.o

In [284]:
X.drop(['DisbursalDate','MaturityDAte','AuthDate'],1,inplace=True)

In [285]:
X.dtypes

ID                       int64
Frequency               object
InstlmentMode           object
LoanStatus              object
PaymentMode             object
Tenure                   int64
AssetCost                int64
AmountFinance          float64
DisbursalAmount        float64
EMI                    float64
ManufacturerID         float64
SupplierID               int64
LTV                    float64
SEX                     object
AGE                    float64
MonthlyIncome          float64
ZiPCODE                float64
Top-up Month            object
DisbursalDate_day        int64
DisbursalDate_month      int64
DisbursalDate_year       int64
MaturityDAte_day         int64
MaturityDAte_month       int64
MaturityDAte_year        int64
AuthDate_day             int64
AuthDate_month           int64
AuthDate_year            int64
dtype: object

In [286]:
X = pd.get_dummies(data=X,columns=['Frequency','InstlmentMode','LoanStatus','PaymentMode','SEX'])

In [287]:
test = X[X['Top-up Month'].isnull()]

In [288]:
X.dropna(inplace=True)

In [157]:
test

,ID,Tenure,AssetCost,AmountFinance,DisbursalAmount,EMI,ManufacturerID,SupplierID,LTV,AGE,...,PaymentMode_Billed,PaymentMode_Direct Debit,PaymentMode_ECS,PaymentMode_ECS Reject,PaymentMode_Escrow,PaymentMode_PDC,PaymentMode_PDC Reject,PaymentMode_SI Reject,SEX_F,SEX_M
0,4,46,480000,365000.0,365000.0,1000.0,1046.0,22354,75.83,50.0,...,0,0,0,0,0,1,0,0,0,1
1,5,45,480000,285000.0,285000.0,9300.0,1046.0,22354,57.44,35.0,...,0,0,0,0,0,1,0,0,0,1
2,6,48,580000,400000.0,400000.0,35800.0,1060.0,65929,68.97,37.0,...,0,0,0,0,0,1,0,0,0,1
3,25,36,725000,500000.0,500000.0,52000.0,1049.0,68597,61.79,55.0,...,1,0,0,0,0,0,0,0,0,1
4,119,48,617000,400000.0,400000.0,35400.0,1049.0,38493,59.09,48.0,...,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14740,143396,35,530016,419616.0,419616.0,14723.0,1046.0,22094,79.17,41.0,...,0,0,0,0,0,1,0,0,0,1
14741,143397,36,595000,446500.0,446500.0,15491.0,2721.0,61196,75.04,41.0,...,0,0,0,0,0,1,0,0,0,1
14742,143398,36,595000,446500.0,446500.0,15491.0,2721.0,61196,75.04,41.0,...,0,0,0,0,0,1,0,0,0,1
14743,143399,12,400000,280000.0,280000.0,25350.0,1046.0,63565,70.00,41.0,...,0,0,0,0,0,1,0,0,0,1


In [110]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score    
def results(y_test,X_test):
    ypred=knn_clf.predict(X_test)
    result = confusion_matrix(y_test, ypred)
    print("Confusion Matrix:")
    print(result)
    result1 = classification_report(y_test, ypred)
    print("Classification Report:")
    print (result1)
    result2 = accuracy_score(y_test,ypred)
    print("Accuracy:",result2)

In [266]:
def fitknn(x_in,y_in):
    test_result = test.copy()
    knn_clf=KNeighborsClassifier(n_neighbors=7)
    knn_clf.fit(x_in,y_in)
    ypred=knn_clf.predict(X_test)
    result = confusion_matrix(y_test, ypred)
    print("Confusion Matrix:")
    print(result)
    result1 = classification_report(y_test, ypred)
    print("Classification Report:")
    print (result1)
    result2 = accuracy_score(y_test,ypred)
    print("Accuracy:",result2)
    pred_result=knn_clf.predict(test.drop('Top-up Month',1))
    test_result['Top-up Month'] = pred_result
    test_result=test_result[['ID','Top-up Month']]
    test_result.to_csv('test_result_knn.csv',index=False)
    

In [297]:
y=X['Top-up Month']
x=X.drop('Top-up Month',1)

In [164]:
test.drop('Top-up Month',1).isnull().sum()

ID                          0
Tenure                      0
AssetCost                   0
AmountFinance               0
DisbursalAmount             0
EMI                         0
ManufacturerID              0
SupplierID                  0
LTV                         0
AGE                         0
MonthlyIncome               7
ZiPCODE                     0
DisbursalDate_day           0
DisbursalDate_month         0
DisbursalDate_year          0
MaturityDAte_day            0
MaturityDAte_month          0
MaturityDAte_year           0
AuthDate_day                0
AuthDate_month              0
AuthDate_year               0
Frequency_BI-Monthly        0
Frequency_Half Yearly       0
Frequency_Monthly           0
Frequency_Quatrly           0
InstlmentMode_Advance       0
InstlmentMode_Arrear        0
LoanStatus_Active           0
LoanStatus_Closed           0
PaymentMode_Auto Debit      0
PaymentMode_Billed          0
PaymentMode_Direct Debit    0
PaymentMode_ECS             0
PaymentMod

In [109]:
rus = RandomUnderSampler(random_state=0)

rus.fit(x,y)

X_resampled, y_resampled = rus.fit_sample(x, y)

In [127]:
X_train,X_test,y_train,y_test = train_test_split(X_resampled, y_resampled,train_size=0.95,random_state=1)

In [257]:
fit(X_train,y_train)

Confusion Matrix:
[[19  6  5  5  3  3  5]
 [ 7 14 11  5  6  3  2]
 [ 8 13 15  3  7  8  3]
 [ 6  9 12  9 10  6  2]
 [ 8  4  6 11 15  8  5]
 [10 10  3  4 14 10  3]
 [26  5  4  2  1  5  3]]
Classification Report:
                   precision    recall  f1-score   support

      > 48 Months       0.23      0.41      0.29        46
     12-18 Months       0.23      0.29      0.26        48
     18-24 Months       0.27      0.26      0.27        57
     24-30 Months       0.23      0.17      0.19        54
     30-36 Months       0.27      0.26      0.27        57
     36-48 Months       0.23      0.19      0.21        54
No Top-up Service       0.13      0.07      0.09        46

         accuracy                           0.23       362
        macro avg       0.23      0.24      0.22       362
     weighted avg       0.23      0.23      0.23       362

Accuracy: 0.23480662983425415


<ipython-input-256-0c7b38d5e779>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Top-up Month'] = test_result


In [259]:
test=test[['ID','Top-up Month']]

test.to_csv('test_result.csv',index=False)

In [295]:
from sklearn import svm

def fitsvm(x_in,y_in):
    test_result = test.copy()
    clf = svm.SVC()
    clf.fit(x_in,y_in)
    ypred=clf.predict(X_test)
    result = confusion_matrix(y_test, ypred)
    print("Confusion Matrix:")
    print(result)
    result1 = classification_report(y_test, ypred)
    print("Classification Report:")
    print (result1)
    result2 = accuracy_score(y_test,ypred)
    print("Accuracy:",result2)
    pred_result=clf.predict(test_result.drop('Top-up Month',1))
    test_result['Top-up Month'] = pred_result
    test_result=test_result[['ID','Top-up Month']]
    test_result.to_csv('test_result_svm.csv',index=False)
    

In [296]:
fitsvm(X_train,y_train)

Confusion Matrix:
[[24  3  1  1  4 11  2]
 [14 11  6  3  4  7  3]
 [15 12 11  4  5 10  0]
 [ 9  4  7 13 12  9  0]
 [11  3  4  5 21 11  2]
 [16  5  3  2 13 12  3]
 [19  7  5  2  2  6  5]]
Classification Report:
                   precision    recall  f1-score   support

      > 48 Months       0.22      0.52      0.31        46
     12-18 Months       0.24      0.23      0.24        48
     18-24 Months       0.30      0.19      0.23        57
     24-30 Months       0.43      0.24      0.31        54
     30-36 Months       0.34      0.37      0.36        57
     36-48 Months       0.18      0.22      0.20        54
No Top-up Service       0.33      0.11      0.16        46

         accuracy                           0.27       362
        macro avg       0.29      0.27      0.26       362
     weighted avg       0.30      0.27      0.26       362

Accuracy: 0.26795580110497236


In [298]:
X_train,X_test,y_train,y_test = train_test_split(x, y,train_size=0.95,random_state=1)

In [299]:
test_res = fitsvm(X_train,y_train)

Confusion Matrix:
[[   0    0    0    0    0    0  423]
 [   0    0    0    0    0    0   63]
 [   0    0    0    0    0    0  137]
 [   0    0    0    0    0    0  186]
 [   0    0    0    0    0    0  166]
 [   0    0    0    0    0    0  188]
 [   0    0    0    0    0    0 5270]]
Classification Report:
                   precision    recall  f1-score   support

      > 48 Months       0.00      0.00      0.00       423
     12-18 Months       0.00      0.00      0.00        63
     18-24 Months       0.00      0.00      0.00       137
     24-30 Months       0.00      0.00      0.00       186
     30-36 Months       0.00      0.00      0.00       166
     36-48 Months       0.00      0.00      0.00       188
No Top-up Service       0.82      1.00      0.90      5270

         accuracy                           0.82      6433
        macro avg       0.12      0.14      0.13      6433
     weighted avg       0.67      0.82      0.74      6433

Accuracy: 0.8192134307477071


C:\Users\rushi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
test_result